In [38]:
import mysql.connector
import pandas as pd

In [39]:
cnx = mysql.connector.connect(user='root', password='TO52',
                              host='localhost',
                              database='clothes_detection')

In [40]:
def construct_query(cat, brand, size):
    query = "SELECT absolute_path FROM clothes NATURAL JOIN category "
    where_category = f"WHERE category = '{cat}' "
    where_brand = ""
    where_size = ""
    first = True
    
    if brand != 'all':
        query = query + "NATURAL JOIN brand "
        for bd in brand:
            if first:
                where_brand = f"AND (brand = '{bd}' "
                first = False
            else:
                where_brand = where_brand + f"OR brand = '{bd}' "
        if len(brand) > 0:
            where_brand = where_brand + ") "

    first = True
    if size != 'all':
        query = query + "NATURAL JOIN size "
        for sz in size:
            if first:
                where_size = f"AND (size = '{sz}' "
                first = False
            else:
                where_size = where_size + f"OR size = '{sz}' "
        if len(size) > 0:
            where_size = where_size + ") "
    
    query = query + where_category + where_brand + where_size + "LIMIT 10"
    
    return query

In [41]:
brand = ['Zara','H&M']
query = construct_query('Tee', brand ,['XS'])
print(query)

SELECT absolute_path FROM clothes NATURAL JOIN category NATURAL JOIN brand NATURAL JOIN size WHERE category = 'Tee' AND (brand = 'Zara' OR brand = 'H&M' ) AND (size = 'XS' ) LIMIT 10


In [42]:
cursor = cnx.cursor()

cat = 'Dress'
brand = ['Zara']
size = ['M','L']

query = construct_query(cat, brand, size)
print(query)

cursor.execute(query)

df = pd.DataFrame(cursor.fetchall())
df.columns = cursor.column_names

print(df)


SELECT absolute_path FROM clothes NATURAL JOIN category NATURAL JOIN brand NATURAL JOIN size WHERE category = 'Dress' AND (brand = 'Zara' ) AND (size = 'M' OR size = 'L' ) LIMIT 10
                                     absolute_path
0   img/Lace_Insert_Bodycon_Dress/img_00000048.jpg
1   img/Lace_Insert_Bodycon_Dress/img_00000029.jpg
2           img/Lace_Halter_Dress/img_00000036.jpg
3           img/Lace_Halter_Dress/img_00000003.jpg
4      img/Lace_Fit_&_Flare_Dress/img_00000036.jpg
5      img/Lace_Fit_&_Flare_Dress/img_00000011.jpg
6      img/Lace_Fit_&_Flare_Dress/img_00000001.jpg
7  img/Lace_Crisscross-Back_Dress/img_00000047.jpg
8     img/Lace_Collar_Shift_Dress/img_00000070.jpg
9     img/Lace_Collar_Shift_Dress/img_00000025.jpg


In [43]:
cnx.close()